In [1]:
import brian2
from brian2 import mV, ms, nS, mS, nF, Hz

import matplotlib.pyplot as plt

In [2]:
dt = 0.001*ms
inv_dt = 1./dt
update = inv_dt

N_e = 800
N_ir = 150
N_if = 50
N_all = N_e + N_ir + N_if

t_init = 500*inv_dt
t_pre = 0*inv_dt
t_stim = 500*inv_dt
t_post = 100*inv_dt
t_all = t_init + t_pre + t_stim + t_post

In [3]:
con_t, con_r, con_p = 0, 0, 0
import random
print(random.getstate())

(3, (4063552922, 1792463263, 688434420, 1710410617, 1594863061, 3853517911, 4147111104, 1271147389, 3953692407, 214444095, 1920394597, 2774906903, 3193859944, 3048321367, 3188451822, 3098286391, 1295402013, 2254026196, 1190836890, 366297305, 4032213169, 2577610530, 4075882562, 4159731495, 1561503099, 5832744, 3552755183, 2659584400, 818149056, 3421190067, 3984590691, 3347348063, 1201239142, 2883358164, 459014682, 613918973, 615008587, 543168859, 938776407, 3775520799, 2212122268, 3709152953, 3377258874, 4266043465, 1583834361, 338476466, 2511840996, 1502822318, 277751138, 3259676630, 4197060001, 262424638, 3213225512, 3452359379, 572365800, 1615707771, 2536691309, 54957530, 2809198194, 3269743486, 2177714699, 2403267435, 2839771334, 3839118544, 694423467, 334229034, 2135117882, 280019497, 2595418537, 2431404086, 3903265381, 2927986437, 1066280045, 1572076217, 3860096363, 3872950509, 2706887057, 2461805398, 2597182240, 666206568, 2934660771, 1796990214, 1845534736, 857095502, 3259612016

In [4]:
ampa_scl = 1.
nmda_e = 0.
nmda_rs = 0.
nmda_fs = 0.
nmda_scl_e = 0.45
nmda_scl_rs = 0.1
nmda_scl_fs = 0.1

gaba = 0
con_t = 0
con_p = 0
con_r = 0
con_if = 0
con_ir = 0
con_i_all = 0
i_del = 0
noise_wt = 0.016
W_scl = 0
decay_gaba = 5.

In [5]:
neuron_model = """
dv/dt = (v - V_L)/tau_m + I_syn/(1000000*uF) : volt
I_syn = I_AMPA + I_NMDA + I_GABA : ampere
"""

neuron_groups = {
    'PC': brian2.NeuronGroup(
        N_e, 
    ),
}

TypeError: __init__() missing 1 required positional argument: 'model'

In [6]:
synapse_AMPANMDA_model = """
dg_AMPA_r/dt = -g_AMPA_r/tau_AMPA_r : siemens (clock-driven)
dg_AMPA_d/dt = -g_AMPA_d/tau_AMPA_d : siemens (clock-driven)
g_AMPA = (g_AMPA_r - g_AMPA_d)/norm_AMPA : siemens (clock-driven)
I_AMPA_post = -scl_AMPA*g_AMPA*(v_pre - V_E) : ampere (summed)
dg_NMDA_r/dt = -g_NMDA_r/tau_NMDA_r : siemens (clock-driven)
dg_NMDA_d/dt = -g_NMDA_d/tau_NMDA_d : siemens (clock-driven)
g_NMDA = (g_NMDA_r - g_NMDA_d)/norm_NMDA : siemens (clock-driven)
I_NMDA_post = -scl_NMDA*g_NMDA*(v_pre - V_E) : ampere (summed)
"""

synapse_AMPANMDA_onpre = """
g_AMPA_r += w
g_AMPA_d += w
g_NMDA_r += w
g_NMDA_d += w
"""

synapse_GABA_model = """
dg_GABA_r/dt = -g_GABA_r/tau_GABA_r : siemens (clock-driven)
dg_GABA_d/dt = -g_GABA_d/tau_GABA_d : siemens (clock-driven)
g_GABA = (g_GABA_r - g_GABA_d)/norm_GABA : siemens (clock-driven)
I_GABA_post = -scl_GABA*g_GABA*(v_pre - V_E) : ampere (summed)
"""

NameError: name 'neuron_groups' is not defined